In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pickle as pkl
from collections import defaultdict,deque,Counter,OrderedDict
from torch.utils.data import DataLoader, Dataset
from torch.optim import lr_scheduler
import os
import time
import copy

from models import LM_latent
from vocab_generic import Vocabulary

In [2]:
!nvidia-smi

Thu May 14 17:48:00 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.129      Driver Version: 410.129      CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P40           Off  | 00000000:05:00.0 Off |                    0 |
| N/A   48C    P0    54W / 250W |   3917MiB / 22919MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P40           Off  | 00000000:08:00.0 Off |                    0 |
| N/A   78C    P0   149W / 250W |  20827MiB / 22919MiB |     97%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [3]:
# general_vocab_size = 10000
# batch_size = 96 #takes about 12gb memory with below config
# hidden_size = 512
# token_embedding_size = 256
# tag_embedding_size = 256
# lstm_layers = 3
# max_sent_len = 60

num_gpus = torch.cuda.device_count()
if num_gpus > 0:
    device = 'cuda:4'
else:
    device = 'cpu'

In [4]:
train_pickle_file = 'train.p'
val_pickle_file = 'val.p'
test_pickle_file = 'test_finance.p'

with open(train_pickle_file,"rb") as f:
    traindict = pkl.load(f)
with open(val_pickle_file,"rb") as f:
    valdict = pkl.load(f)
with open(test_pickle_file,"rb") as f:
    testdict = pkl.load(f)

In [5]:
alltraintoks = set([tup[0] for tup in traindict['tagged_words']])
allvaltoks = Counter([tup[0] for tup in valdict['tagged_words']])
#alltesttoks = Counter([tup[0] for tup in testdict['tagged_words']])

In [6]:
with open('tagset.txt') as a:
    alltags = a.read()

alltags = alltags.split('\n')    
alltags = alltags + ['UNKNOWN']
alltags = set(alltags)

tag2id = defaultdict(int)
id2tag = defaultdict(str)
for i, tag in enumerate(alltags):
    tag2id[tag] = i
    id2tag[i] = tag
    
UNKNOWN_TAG = tag2id['UNKNOWN']
PAD_TAG_ID = -51

In [7]:
class POSDataset(Dataset):
    def __init__(self, instanceDict, vocab, tag2id, id2tag, max_sent_len=60,test=False):
        if not test:
            self.root = instanceDict['tagged_sents']
            self.sents = [[s[0] for s in sentences] for sentences in self.root]
        else:
            self.root = instanceDict
            self.sents = [[s[0] for s in sentences] for sentences in self.root]
        self.vocab = vocab
        self.tag2id = tag2id
        self.id2tag = id2tag
        
        
        self.input_sents = []
        self.output_sents = []
        self.tags = []
        self.priors = []
        for sample in self.sents:
            
            if max_sent_len == None:
                mlength = len(sample)
            else:
                mlength = max_sent_len
                
            newsample = [Vocabulary.BOS] + sample[:mlength] + [Vocabulary.EOS]
            input_toks = self.vocab.encode_token_seq(newsample[:-1])
            output_toks = [self.vocab.encode_token_seq_tag(newsample[1:], self.id2tag[tagid]) for tagid in self.id2tag]
            self.input_sents.append(input_toks)
            self.output_sents.append(output_toks)
            self.priors.append([[self.vocab.tag_cntr[t][word]+1 for t in self.vocab.alltags] for word in newsample[1:]])
            
        for sentences in self.root:
            
            if max_sent_len == None:
                mlength = len(sentences)
            else:
                mlength = max_sent_len
            
            outputsample = sentences[:mlength] + [(Vocabulary.EOS, 'UNKNOWN')]
            outputsample = [self.tag2id[tup[1]] if tup[1] in self.tag2id else self.tag2id['UNKNOWN'] for tup in outputsample]
            self.tags.append(outputsample)
    
    def __len__(self):
        return len(self.root)
    
    def __getitem__(self,idx):
        target_tensor = torch.as_tensor(self.tags[idx], dtype=torch.long)
        input_tensor = torch.as_tensor(self.input_sents[idx], dtype=torch.long)
        output_tensor = torch.as_tensor(self.output_sents[idx], dtype=torch.long)
        prior_tensor_num = torch.as_tensor(self.priors[idx], dtype=torch.float)
        prior_tensor_denom = torch.sum(prior_tensor_num, dim=-1)
        prior_tensor = prior_tensor_num/prior_tensor_denom.view(-1,1)
        return (input_tensor, output_tensor, target_tensor, prior_tensor)

In [8]:
def pad_list_of_tensors(list_of_tensors, pad_token):
    max_length = max([t.size(-1) for t in list_of_tensors])
    padded_list = []
    for t in list_of_tensors:
        padding = torch.zeros(list(t.shape)[:-1] + [max_length - t.size(-1)], dtype=torch.long) + pad_token
        padded_tensor = torch.cat([t, padding], dim = -1)
        padded_list.append(padded_tensor)
    padded_tensor = torch.stack(padded_list)
    return padded_tensor

def pad_list_of_priors(list_of_tensors,pad_token):
    max_length = max([t.size(0) for t in list_of_tensors])
    padded_list = []
    for t in list_of_tensors:
        padding = torch.zeros((max_length - t.size(0),t.shape[1]), dtype=torch.float) + pad_token
        padded_tensor = torch.cat([t, padding], dim = 0)
        padded_list.append(padded_tensor)
    padded_tensor = torch.stack(padded_list)
    return padded_tensor


def pad_collate_fn_pos(batch):
    # batch is a list of sample tuples
    input_list = [s[0] for s in batch]
    target_list = [s[1] for s in batch]
    target_labels = [s[2] for s in batch]
    target_priors = [s[3] for s in batch]
    pad_token_input = 2
    pad_token_output = Vocabulary.PADTOKEN_FOR_TAGVOCAB
    pad_token_tags = PAD_TAG_ID
    pad_prior = 1/len(vocab.alltags) 
    input_tensor = pad_list_of_tensors(input_list, pad_token_input)
    target_tensor = pad_list_of_tensors(target_list, pad_token_output)
    target_labels = pad_list_of_tensors(target_labels, pad_token_tags)
    target_priors = pad_list_of_priors(target_priors, pad_prior)
    return input_tensor, target_tensor, target_labels,target_priors

In [9]:
def log_sum_exp(value, dim=None, keepdim=False):
    """Numerically stable implementation of the operation

    value.exp().sum(dim, keepdim).log()
    """
    # TODO: torch.max(value, dim=None) threw an error at time of writing
    if dim is not None:
        m, _ = torch.max(value, dim=dim, keepdim=True)
        value0 = value - m
        if keepdim is False:
            m = m.squeeze(dim)
        return m + torch.log(torch.sum(torch.exp(value0),
                                       dim=dim, keepdim=keepdim))
    else:
        m = torch.max(value)
        sum_exp = torch.sum(torch.exp(value - m))
        if isinstance(sum_exp, Number):
            return m + math.log(sum_exp)
        else:
            return m + torch.log(sum_exp)

def latent_loss(outputs, target, priors=None,device=device):
    """Numerically stable implementation of the language modeling loss

    """
    #target dim # btchsize x numtags x sentLen
    tag_logits = outputs[0] #btchsize x sentlen x numtags
    word_dist_logits = outputs[1] #list #for jth tag -> batch_size, sent_len, j_vocab_size
    
    numtags = len(word_dist_logits)
    btchSize = tag_logits.shape[0]
    sentLen = tag_logits.shape[1]
    
    #calculate loss for tags
    crossEntropy_tag = nn.CrossEntropyLoss(reduction='none')
    taglogitloss = [-crossEntropy_tag(tag_logits.transpose(1,2), torch.zeros((btchSize, sentLen), dtype=torch.long, device=device) + j) for j in range(numtags)]
    
    #calculate loss for words
    ignore_mask = ((target == Vocabulary.TOKEN_NOT_IN_TAGVOCAB) | (target == Vocabulary.PADTOKEN_FOR_TAGVOCAB))
    target_with_ignore = target.clone()
    target_with_ignore[ignore_mask] = -100
    crossEntropy_word = nn.CrossEntropyLoss(reduction='none', ignore_index=-100)
    wordlogitloss = [-crossEntropy_word(word_logit.transpose(1,2), target_with_ignore[:, j, :])  for j, word_logit in enumerate(word_dist_logits)]
    
    taglogitloss = torch.stack(taglogitloss)
    wordlogitloss = torch.stack(wordlogitloss)
    totalloss = taglogitloss + wordlogitloss
    
    #0 loss for a tag if output word is not present in tag's vocab
    outofvocab_mask = (torch.transpose(target, 0, 1) == Vocabulary.TOKEN_NOT_IN_TAGVOCAB)
    totalloss[outofvocab_mask] = float('-inf')

    finalLoss = -log_sum_exp(totalloss, dim=0)
    
    #mask the loss from tokens, if the output token is not present in even single tag category
    presentInZeroTagMask = torch.all((torch.transpose(target, 1, 2) == Vocabulary.TOKEN_NOT_IN_TAGVOCAB), dim=-1)
    #mask the loss of padding tokens
    paddingMask = (target[:, 0, :] == Vocabulary.PADTOKEN_FOR_TAGVOCAB)
    tokenContributingToZeroLoss = (presentInZeroTagMask | paddingMask)
    num_useful_tokens = (~tokenContributingToZeroLoss).sum().item()
    
    if priors is not None:
        kld_loss = F.kl_div(F.log_softmax(tag_logits,dim=-1),priors,reduction='sum')
    else:
        kld_loss = 0
    #print(kld_loss)
    #print(torch.sum(finalLoss[~tokenContributingToZeroLoss]))
    
    return torch.sum(finalLoss[~tokenContributingToZeroLoss])+(lamda)*kld_loss, num_useful_tokens

In [10]:
import torch
import torch.nn as nn
from torch import cat
import torch.nn.init as init
import math


class LM_latent_bidirectional(nn.Module):
    def __init__(self, vocab_size, tag_wise_vocab_size, hidden_size, token_embedding_size, tag_embedding_size, lstm_layers=1, dropout_p=0.1):

        super(LM_latent_bidirectional, self).__init__()

        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.token_embedding_size = token_embedding_size
        self.tag_embedding_size = tag_embedding_size
        self.num_tags = len(tag_wise_vocab_size)
        self.tag_wise_vocabsize = tag_wise_vocab_size
        
        self.token_embedding = nn.Embedding(self.vocab_size, self.token_embedding_size)
        self.lstm = nn.LSTM(self.token_embedding_size, self.hidden_size, num_layers = lstm_layers, batch_first = True,bidirectional=True)
        self.tag_linear = nn.Linear(self.tag_embedding_size, self.num_tags)
        self.lower_hidden = nn.Linear(2*self.hidden_size, self.tag_embedding_size)
        self.tag_projections = nn.ModuleList([nn.Linear(2*self.hidden_size, self.tag_wise_vocabsize[i]) for i in range(self.num_tags)])
        self.dropout = nn.Dropout(p=dropout_p)
        
    def forward(self,input_seq):
        
        batch_size,sent_len = input_seq.shape[0],input_seq.shape[1]
        
        embeddings = self.token_embedding(input_seq) #batch_size, sent_len, embed_size
        embeddings = self.dropout(embeddings)
        h, _ = self.lstm(embeddings) #batch_size, sent_len, 2*hidden_size
        #print(h.shape)
        
        h_lower = self.lower_hidden(h) #batch_size , sent_len, 100
        tag_logits = self.tag_linear(h_lower) #batch_size, sent_len, num_tags
        
        word_distribution_logits = [self.tag_projections[i](h) for i in range(self.num_tags)] #for ith tag -> batch_size, sent_len, i_vocab_size       
        return tag_logits, word_distribution_logits

In [11]:
def train_model(model, criterion, optimizer, scheduler, device, checkpoint_path, f, verbIter, hyperparams, num_epochs=25):
    metrics_dict = {}
    metrics_dict["train"] = {}
    metrics_dict["valid"] = {}
    metrics_dict["train"]["loss"] = {}
    metrics_dict["train"]["loss"]["epochwise"] = []
    metrics_dict["train"]["loss"]["stepwise"] = []
    metrics_dict["valid"]["loss"] = []
    metrics_dict["valid"]["tokenacc"] = []
    metrics_dict["valid"]["sentacc"] = []
        
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 9999999999999999

    for epoch in range(num_epochs):
        f.write('Epoch {}/{} \n'.format(epoch, num_epochs - 1))
        f.write('-' * 10)
        f.write('\n')
        f.flush()
        
        #train phase
        scheduler.step()
        model.train()
        
        running_loss = 0.0
        n_samples = 0
        non_pad_tokens_cache = 0

        end = time.time()
        
        for batch_num, (inputs, target, labels,priors) in enumerate(dataloaders["train"]):
            
            data_time = time.time() - end
            inputs = inputs.to(device)
            target = target.to(device)
            priors = priors.to(device)
            
            # zero the parameter gradients
            optimizer.zero_grad()
            batchSize = inputs.size(0)
            n_samples += batchSize

            # forward
            # track history if only in train
            forward_start_time  = time.time()

            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                loss, useful_tokens = criterion(outputs, target, priors,device)
                
                # statistics
                running_loss += loss.item()

                non_pad_tokens_cache += useful_tokens

                loss /= useful_tokens

                loss.backward()
                optimizer.step()
            
            forward_time = time.time() - forward_start_time

            if batch_num % verbIter == 0:
                # Metrics
                epoch_loss = running_loss / non_pad_tokens_cache

                f.write('Train Loss: {:.4f} \n'.format(epoch_loss))
                f.write('Full Batch time: {} , Data load time: {} , Forward time: {}\n'.format(time.time() - end, data_time, forward_time))
                f.flush()

                metrics_dict["train"]["loss"]["stepwise"].append(epoch_loss)

            end = time.time()
        
        # Metrics
        epoch_loss = running_loss / non_pad_tokens_cache
        f.write('Train Loss: {:.4f} \n'.format(epoch_loss))
        f.flush()
        metrics_dict["train"]["loss"]["epochwise"].append(epoch_loss)

        
        #val phase
        epoch_loss, tokenaccuracy, sentaccuracy = evaluate(model, criterion, device, dataloaders["valid"])
        f.write('Validation Loss: {:.4f}, Perplexity: {},  TokenAccuracy: {}, SentAccuracy: {} \n'.format(epoch_loss, perplexity(epoch_loss), tokenaccuracy, sentaccuracy))
        f.flush()
        metrics_dict["valid"]["loss"].append(epoch_loss)
        metrics_dict["valid"]["tokenacc"].append(tokenaccuracy)
        metrics_dict["valid"]["sentacc"].append(sentaccuracy)
        
            
        # deep copy the model
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            best_model_wts = copy.deepcopy(model.state_dict())
                
            torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'full_metrics': metrics_dict,
            'hyperparams': hyperparams
            }, '%s/net_epoch_%d.pth' % (checkpoint_path, epoch))

    time_elapsed = time.time() - since
    f.write('Training complete in {:.0f}m {:.0f}s \n'.format(time_elapsed // 60, time_elapsed % 60))
    f.write('Best val loss: {:4f} \n'.format(best_loss))
    f.flush()

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [12]:
def getTagPredictions(tag_logits, targets):
    #targets dim # btchsize x numtags x sentLen
    btch_size = tag_logits.shape[0]
    sent_len = tag_logits.shape[1]
    num_tags = tag_logits.shape[2]

#     tag_logits_cloned = tag_logits.clone()
#     outofvocab_mask = (torch.transpose(targets, 1, 2) == Vocabulary.TOKEN_NOT_IN_TAGVOCAB)
#     tag_logits_cloned[outofvocab_mask] = float('-inf')
    predictions = torch.max(tag_logits, dim=-1).indices #btchsize x sentlen
    return predictions


def perplexity(avg_epoch_loss):
    return 2**(avg_epoch_loss/np.log(2))

def getTokenAccuracy(tag_logits, labels, targets):
    #tag_logits ->  btchsize x sentlen x numtags
    #labels -> btchsize x sentlen
    predictions = getTagPredictions(tag_logits, targets) #btchsize x sentlen
    mask = ((labels != UNKNOWN_TAG) & (labels != PAD_TAG_ID))
    num = (predictions[mask] == labels[mask]).sum().item()
    den = labels[mask].shape[0]
    return num, den

def getSentenceAccuracy(tag_logits, labels, targets):
    #tag_logits ->  btchsize x sentlen x numtags
    #labels -> btchsize x sentlen
    predictions = getTagPredictions(tag_logits, targets) #btchsize x sentlen
    mask = ((labels != UNKNOWN_TAG) & (labels != PAD_TAG_ID))
    
    sentCount = 0
    for i in range(tag_logits.shape[0]):
        maski = mask[i]
        labeli = labels[i,:]
        predictioni = predictions[i,:]
        result = torch.equal(labeli[maski], predictioni[maski])
        sentCount += result*1
    
    return sentCount, tag_logits.shape[0]


def evaluate(model, criterion, device, validation_loader):  
    model.eval()   # Set model to evaluate mode
    running_loss = 0.0
    running_word = 0
    running_sent = 0
    total_words = 0
    total_sents = 0
    n_samples = 0
    non_pad_tokens_cache = 0

    # Iterate over data.
    for batch_num, (inputs, targets, labels, priors) in enumerate(validation_loader):

        inputs = inputs.to(device)
        targets = targets.to(device)
        labels = labels.to(device)
        priors = priors.to(device)

        batchSize = inputs.size(0)
        n_samples += batchSize

        outputs = model(inputs)
        loss, useful_tokens = criterion(outputs, targets, None,device)
                    
        # statistics
        running_loss += loss.item()
        num, den = getTokenAccuracy(0.4*F.softmax(outputs[0])+0.6*priors, labels, targets)
        running_word += num
        total_words += den
        num, den = getSentenceAccuracy(0.4*F.softmax(outputs[0])+0.6*priors, labels, targets)
        running_sent += num
        total_sents += den

        non_pad_tokens_cache += useful_tokens

    # Metrics
    epoch_loss = running_loss / non_pad_tokens_cache
    tokenaccuracy = running_word/total_words
    sentaccuracy = running_sent/total_sents
    return epoch_loss, tokenaccuracy, sentaccuracy

In [13]:
general_vocab_size = 10000
tag_vocab_size = 25000
batch_size = 96 #takes about 12gb memory with below config
hidden_size = 512
token_embedding_size = 256
tag_embedding_size = 256
lstm_layers = 2
max_sent_len = 60
dropout = 0.1
wtdecay = 0
lamda = 10

In [14]:
vocab = Vocabulary(traindict, traindict['tagged_sents'], general_vocab_size, tag_vocab_size, alltags)
tag_wise_vocabsize = dict([(tag2id[tup[0]], tup[1][2]) for tup in vocab.tag_specific_vocab.items()])

In [15]:
datasets = {}
dataloaders = {}

datasets["train"] = POSDataset(traindict, vocab, tag2id, id2tag,test=False)
datasets["valid"] = POSDataset(valdict, vocab, tag2id, id2tag, None,test=False)
datasets["test"] = POSDataset(testdict, vocab, tag2id, id2tag, None,test=True)

dataloaders["train"] = DataLoader(datasets["train"], batch_size=batch_size, shuffle=True, collate_fn=pad_collate_fn_pos, pin_memory=True)
dataloaders["valid"] = DataLoader(datasets["valid"], batch_size=batch_size, shuffle=False, collate_fn=pad_collate_fn_pos, pin_memory=True)
dataloaders["test"] = DataLoader(datasets["test"], batch_size=batch_size, shuffle=False, collate_fn=pad_collate_fn_pos, pin_memory=True)

In [22]:
options = {"vocab":vocab, "hidden_size": hidden_size, "token_embedding":token_embedding_size, 
           "tag_emb_size":tag_embedding_size, "lstmLayers": lstm_layers, "tagtoid":tag2id}

lr = 0.01
stepsize = 5
epochs = 30
outfolder = 'models_priors/embedding{}hidden{}layers{}dropout{}wtdecay{}lambda{}'.format(token_embedding_size,
                                                                          hidden_size,lstm_layers,dropout,wtdecay,lamda)

In [23]:
import os
if not os.path.exists(outfolder):
    os.makedirs(outfolder)

In [ ]:
model = LM_latent_bidirectional(vocab.vocab_size, tag_wise_vocabsize, hidden_size, token_embedding_size, tag_embedding_size, lstm_layers,dropout).to(device)
criterion = latent_loss
model_parameters = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.Adam(model_parameters, lr=lr,weight_decay=wtdecay)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=stepsize, gamma=0.1)
f = open(os.path.join(outfolder, 'training_logs.txt'), 'w+')
bst_model = train_model(model, criterion, optimizer, exp_lr_scheduler, device, outfolder, f, 50, options, epochs)

/gpfs/data/geraslab/ashwin/miniconda3/envs/dl20/lib/python3.7/site-packages/ipykernel_launcher.py:69: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/gpfs/data/geraslab/ashwin/miniconda3/envs/dl20/lib/python3.7/site-packages/ipykernel_launcher.py:72: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [18]:
model_pth = 'models_priors/embedding256hidden512layers2dropout0.1wtdecay0/net_epoch_1.pth'
checkpoint = torch.load(model_pth)
#vocab = checkpoint['hyperparams']['vocab']
#tag_wise_vocabsize = dict([(tag2id[tup[0]], tup[1][2]) for tup in vocab.tag_specific_vocab.items()])
#model = LM_latent_bidirectional(vocab.vocab_size, tag_wise_vocabsize, hidden_size, token_embedding_size, tag_embedding_size, lstm_layers,dropout).to(device)

model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [19]:
def evaluate_alpha(model, criterion, device, validation_loader, alpha):  
    model.eval()   # Set model to evaluate mode
    running_loss = 0.0
    running_word = 0
    running_sent = 0
    total_words = 0
    total_sents = 0
    n_samples = 0
    non_pad_tokens_cache = 0

    # Iterate over data.
    for batch_num, (inputs, targets, labels, priors) in enumerate(validation_loader):

        inputs = inputs.to(device)
        targets = targets.to(device)
        labels = labels.to(device)
        priors = priors.to(device)

        batchSize = inputs.size(0)
        n_samples += batchSize

        outputs = model(inputs)
#         loss, useful_tokens = criterion(outputs, targets, None,device)
                    
        # statistics
#         running_loss += loss.item()
        num, den = getTokenAccuracy(alpha*F.softmax(outputs[0])+(1-alpha)*priors, labels, targets)
        running_word += num
        total_words += den
        num, den = getSentenceAccuracy(alpha*F.softmax(outputs[0])+(1-alpha)*priors, labels, targets)
        running_sent += num
        total_sents += den

#         non_pad_tokens_cache += useful_tokens

    # Metrics
#     epoch_loss = running_loss / non_pad_tokens_cache
    tokenaccuracy = running_word/total_words
    sentaccuracy = running_sent/total_sents
    return tokenaccuracy, sentaccuracy

In [25]:
evaluate_alpha(model, criterion, device, dataloaders["valid"],0.2)

/gpfs/data/geraslab/ashwin/miniconda3/envs/dl20/lib/python3.7/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/gpfs/data/geraslab/ashwin/miniconda3/envs/dl20/lib/python3.7/site-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


(0.7075848584828136, 0.9271581134250201, 0.2055364573909897)

In [11]:
PATH = 'models_priors/embedding256hidden512layers2dropout0.1wtdecay0/net_epoch_1.pth'
#BEST_PATH = './Models/hid_512_emb_64/net_best_weights.pth'

In [12]:
checkpoint = torch.load(PATH)
tag2id = checkpoint['hyperparams']['tagtoid']
vocab = checkpoint['hyperparams']['vocab']
batch_size = 10
hidden_size = checkpoint['hyperparams']['hidden_size']
token_embedding_size = checkpoint['hyperparams']['token_embedding']
tag_embedding_size = checkpoint['hyperparams']['tag_emb_size']
lstm_layers = checkpoint['hyperparams']['lstmLayers']

In [13]:
id2tag = defaultdict(str)
for tag, i in tag2id.items():
    id2tag[i] = tag
    
UNKNOWN_TAG = tag2id['UNKNOWN']
PAD_TAG_ID = -51

In [14]:
datasets = {}
dataloaders = {}

#datasets["train"] = POSDataset(traindict, vocab, tag2id, id2tag,test=False)
#datasets["valid"] = POSDataset(valdict, vocab, tag2id, id2tag, None,test=False)
datasets["test"] = POSDataset(testdict, vocab, tag2id, id2tag, None,test=True)

#dataloaders["train"] = DataLoader(datasets["train"], batch_size=batch_size, shuffle=True, collate_fn=pad_collate_fn_pos, pin_memory=True)
#dataloaders["valid"] = DataLoader(datasets["valid"], batch_size=batch_size, shuffle=False, collate_fn=pad_collate_fn_pos, pin_memory=True)
#dataloaders["test"] = DataLoader(datasets["test"], batch_size=2, shuffle=False, collate_fn=pad_collate_fn_pos, pin_memory=True)

In [15]:
a,b,c,d = datasets['test'][0]
a.shape,b.shape,c.shape,d.shape

(torch.Size([32]),
 torch.Size([47, 32]),
 torch.Size([32]),
 torch.Size([32, 47]))

In [16]:
dataloaders["test"] = DataLoader(datasets["test"], batch_size=2, shuffle=False, collate_fn=pad_collate_fn_pos, pin_memory=True)

In [17]:
tag_wise_vocabsize = dict([(tag2id[tup[0]], tup[1][2]) for tup in vocab.tag_specific_vocab.items()])

In [18]:
model = LM_latent_bidirectional(vocab.vocab_size, tag_wise_vocabsize, hidden_size, token_embedding_size, tag_embedding_size, lstm_layers).to(device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

LM_latent_bidirectional(
  (token_embedding): Embedding(10004, 256)
  (lstm): LSTM(256, 512, num_layers=2, batch_first=True, bidirectional=True)
  (tag_linear): Linear(in_features=256, out_features=47, bias=True)
  (lower_hidden): Linear(in_features=1024, out_features=256, bias=True)
  (tag_projections): ModuleList(
    (0): Linear(in_features=1024, out_features=4, bias=True)
    (1): Linear(in_features=1024, out_features=3, bias=True)
    (2): Linear(in_features=1024, out_features=36, bias=True)
    (3): Linear(in_features=1024, out_features=102, bias=True)
    (4): Linear(in_features=1024, out_features=9150, bias=True)
    (5): Linear(in_features=1024, out_features=34, bias=True)
    (6): Linear(in_features=1024, out_features=2, bias=True)
    (7): Linear(in_features=1024, out_features=152, bias=True)
    (8): Linear(in_features=1024, out_features=605, bias=True)
    (9): Linear(in_features=1024, out_features=9, bias=True)
    (10): Linear(in_features=1024, out_features=15, bias=True

In [22]:
criterion = latent_loss
evaluate_alpha(model, criterion, device, dataloaders["test"],1)

/gpfs/data/geraslab/ashwin/miniconda3/envs/dl20/lib/python3.7/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/gpfs/data/geraslab/ashwin/miniconda3/envs/dl20/lib/python3.7/site-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


(0.5385610471587509, 0.0)

In [19]:
!nvidia-smi

Thu May 14 14:04:05 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.129      Driver Version: 410.129      CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P40           Off  | 00000000:05:00.0 Off |                    0 |
| N/A   33C    P0    51W / 250W |   2953MiB / 22919MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P40           Off  | 00000000:08:00.0 Off |                    0 |
| N/A   36C    P0    51W / 250W |  22703MiB / 22919MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T